# Generate the original model

In [1]:
from pathlib import Path

import numpy as np

from shamo import FEModel
from shamo import MeshConfig

model = FEModel("example_dipole", str(Path("./derivatives")))

In [2]:
# Generate the parameters
labels = np.ones((11, 11, 11), dtype=np.uint8)
labels[3:-3, 3:-3, 3:-3] = 2
tissues = ["a", "b"]
affine = np.diag([1, 1, 1, 1])
mesh_config = MeshConfig(facet_distance=0.1, cell_size=0.75)

# Generate the mesh
model.mesh_from_labels(labels, tissues, affine, mesh_config=mesh_config)
print("Mesh generated.")

Mesh generated.


/home/martin/CRC/projects/shamo/.venv/lib/python3.7/site-packages/numpy/ctypeslib.py:523: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return array(obj, copy=False)


In [3]:
# Define the sensors
coordinates = [(x, y, z) for x in (0, 10) for y in (0, 10) for z in (0, 10)]
sensors = {chr(ord("A") + i): c for i, c in enumerate(coordinates)}

# Add the sensors
model.add_sensors(sensors, "a")
print("Sensors placed.")

Sensors placed.


In [4]:
model.save()

# Generate leadfield matrix

In [5]:
from shamo import EEGForwardProblem

problem = EEGForwardProblem()

In [6]:
problem.set_electrical_conductivity("a", 1.0)
problem.set_electrical_conductivity("b", 0.5)

{'regions_of_interest': [],
 'markers': [],
 'electrical_conductivity': {'a': {'value': 1.0, 'anisotropy': ''},
  'b': {'value': 0.5, 'anisotropy': ''}},
 'reference': ''}

In [7]:
problem.set_reference("A")

{'regions_of_interest': [],
 'markers': [],
 'electrical_conductivity': {'a': {'value': 1.0, 'anisotropy': ''},
  'b': {'value': 0.5, 'anisotropy': ''}},
 'reference': 'A'}

In [8]:
problem.add_region_of_interest("b")

{'regions_of_interest': ['b'],
 'markers': [],
 'electrical_conductivity': {'a': {'value': 1.0, 'anisotropy': ''},
  'b': {'value': 0.5, 'anisotropy': ''}},
 'reference': 'A'}

In [9]:
leadfield = problem.solve("forward_model", 
                          model.path, model)

# Generate the dipole model

In [10]:
dipole_model = FEModel("dipole_model", str(Path(model.path)))

In [11]:
# Generate the mesh
affine = np.diag([1, 1, 1, 1])
dipole_model.mesh_from_labels(labels, tissues, affine, mesh_config=mesh_config)
print("Mesh generated.")

Mesh generated.


/home/martin/CRC/projects/shamo/.venv/lib/python3.7/site-packages/numpy/ctypeslib.py:523: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return array(obj, copy=False)


### Add dipoles

In [12]:
from shamo import EEGSource

source = EEGSource((5, 5, 5), (1, 0, 0), 1e-6)
sources = [source]
dipole_model.add_sources(sources, "b", lc=1e-4)

In [13]:
# Add the sensors
dipole_model.add_sensors(sensors, "a")
print("Sensors placed.")

Sensors placed.


In [14]:
dipole_model.save()

# Simulate the dipoles

In [15]:
from shamo import EEGSimulationProblem

sim = EEGSimulationProblem()

In [16]:
sim.set_electrical_conductivity("a", 1.0)
sim.set_electrical_conductivity("b", 0.5)

{'regions_of_interest': [],
 'markers': [],
 'electrical_conductivity': {'a': {'value': 1.0, 'anisotropy': ''},
  'b': {'value': 0.5, 'anisotropy': ''}},
 'reference': '',
 'sources': []}

In [17]:
sim.set_reference("A")

{'regions_of_interest': [],
 'markers': [],
 'electrical_conductivity': {'a': {'value': 1.0, 'anisotropy': ''},
  'b': {'value': 0.5, 'anisotropy': ''}},
 'reference': 'A',
 'sources': []}

In [18]:
sim.add_source(source)

{'regions_of_interest': [],
 'markers': [],
 'electrical_conductivity': {'a': {'value': 1.0, 'anisotropy': ''},
  'b': {'value': 0.5, 'anisotropy': ''}},
 'reference': 'A',
 'sources': [{'coordinates': (0.005, 0.005, 0.005),
   'orientation': (1, 0, 0),
   'value': 1e-06,
   'unit_orientation': (1.0, 0.0, 0.0)}]}

In [19]:
simulation = sim.solve("simulation", dipole_model.path, dipole_model)

In [20]:
leadfield.evaluate_for_sources(sources)

{'B': 0.001085688720703125,
 'C': 2.1797139942646026e-07,
 'D': 0.000728918212890625,
 'E': -0.008647761718749999,
 'F': -0.009427546875,
 'G': -0.008411673828124999,
 'H': -0.009410140624999999}

In [21]:
simulation.recordings

{'B': 4.4724631184517305e-05,
 'C': -2.7292777773801902e-05,
 'D': -2.6684757254591407e-05,
 'E': -0.0094058649498176,
 'F': -0.00940383219053928,
 'G': -0.00943844265710228,
 'H': -0.009399378927783503}